## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-18-54-55 +0000,nypost,Queens pol seeks overhaul of woke gas-to-elect...,https://nypost.com/2026/01/24/us-news/queens-p...
1,2026-01-24-18-52-35 +0000,bbc,Trump says UK soldiers in Afghanistan 'among g...,https://www.bbc.com/news/articles/c3edwx37pd9o...
2,2026-01-24-18-51-00 +0000,wsj,Border Patrol Agents Fatally Shoot Man in Minn...,https://www.wsj.com/us-news/federal-agents-sho...
3,2026-01-24-18-48-31 +0000,wapo,Live updates: Man fatally shot by federal agen...,https://www.washingtonpost.com/nation/2026/01/...
4,2026-01-24-18-48-31 +0000,wapo,Minneapolis council leader says ICE should leave,https://www.washingtonpost.com


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2447/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,43
31,ice,17
103,china,14
102,canada,11
57,russia,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
54,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...,91
27,2026-01-24-17-20-53 +0000,nypost,Here’s a look at Arctic drones that can patrol...,https://nypost.com/2026/01/24/us-news/heres-a-...,88
14,2026-01-24-18-30-00 +0000,wsj,Trump Threatens New Tariffs on Canada Over China,https://www.wsj.com/politics/trump-threatens-n...,86
48,2026-01-24-15-14-19 +0000,nypost,Trump threatens Canada with 100% tariff after ...,https://nypost.com/2026/01/24/us-news/trump-th...,85
39,2026-01-24-16-27-18 +0000,bbc,Trump threatens Canada with 100% tariffs over ...,https://www.bbc.com/news/articles/cy4qww3w72lo...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
54,91,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...
35,47,2026-01-24-16-56-55 +0000,bbc,Two days of peace talks end as Russia continue...,https://www.bbc.com/news/articles/cwyr6p30054o...
27,45,2026-01-24-17-20-53 +0000,nypost,Here’s a look at Arctic drones that can patrol...,https://nypost.com/2026/01/24/us-news/heres-a-...
6,38,2026-01-24-18-47-27 +0000,startribune,Live: Federal agents shot and killed Minneapol...,https://www.startribune.com/ice-raids-minnesot...
49,35,2026-01-24-15-01-46 +0000,nypost,Trump reveals to The Post ‘on land’ drug strik...,https://nypost.com/2026/01/24/us-news/trump-re...
32,30,2026-01-24-17-06-00 +0000,wsj,China’s Xi Places His Top General Under Invest...,https://www.wsj.com/world/china/chinas-xi-plac...
69,27,2026-01-24-13-04-02 +0000,cbc,Thousands of flights cancelled across U.S. as ...,https://www.cbc.ca/news/world/us-winter-storm-...
230,26,2026-01-23-19-56-34 +0000,latimes,California's fight with offshore oil firm esca...,https://www.latimes.com/california/story/2026-...
89,25,2026-01-24-10-50-00 +0000,startribune,NFL championship game picks: NFL’s final four ...,https://www.startribune.com/nfl-championship-g...
51,25,2026-01-24-14-36-53 +0000,nypost,NYC dogs recreate celeb looks from the Golden ...,https://nypost.com/2026/01/24/entertainment/ny...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
